In [1]:
import tensorflow as tf
import numpy as np

from matplotlib import pyplot as plt
%matplotlib inline

from random import randint

from load_data import gen_data,load_data,standardize

In [2]:
tf.reset_default_graph()

num_steps = 70


input_num = 5
output_num = 1

state_size = 100

num_layers = 20

def lrelu(x, name, leak=0.2): 
  #  return tf.maximum(x, leak * x, name=name) 
    return tf.maximum(x, 0, name=name) 

In [3]:
tf.reset_default_graph()
x = tf.placeholder(tf.float32, [None, num_steps,input_num], name='input_placeholder')
y = tf.placeholder(tf.float32, [None], name='labels_placeholder')

keep_prob = tf.placeholder_with_default(1.0, shape=())



x_reshaped = tf.reshape(x,[tf.shape(x)[0],x.shape[1],1,x.shape[2]])



#conv1 = lrelu(tf.layers.conv2d(x_reshaped,name="conv1", filters = 8, 
#                         kernel_size = [4,1],strides = [1,1],padding = "valid", activation = None),"conv1_lrelu")

#conv2 = lrelu(tf.layers.conv2d(conv1, name="conv2", filters = 16, 
#                         kernel_size = [4,1],strides = [1,1],padding = "valid", activation = None),"conv2_lrelu")

#pool1 = tf.layers.max_pooling2d(inputs=conv2, name = "pool1", pool_size=[2, 1], strides=[2,1])

#conv3 = lrelu(tf.layers.conv2d(pool1, name="conv3", filters = 8, 
#                         kernel_size = [4,1],strides = [1,1],padding = "valid", activation = None),"conv3_lrelu")
#bn3 = tf.contrib.layers.batch_norm(conv3,is_training=is_training, scope='bn3')

#conv4 = lrelu(tf.layers.conv2d(conv3, name="conv4", filters = 16, 
#                         kernel_size = [4,1],strides = [1,1],padding = "valid", activation = None),"conv4_lrelu")
#bn4 = tf.contrib.layers.batch_norm(conv4,is_training=is_training, scope='bn4')


#pool2 = tf.layers.max_pooling2d(inputs=bn4, name = "pool2", pool_size=[2, 1], strides=[1,1])
rnn_inputs = tf.squeeze(x_reshaped,axis=2)


In [4]:
print(x_reshaped)
#print(conv1)
#print(conv2)
#print(pool1)


Tensor("Reshape:0", shape=(?, 70, 1, 5), dtype=float32)


In [5]:



cells = []
for i in range(0, num_layers):
    cell = tf.contrib.rnn.LSTMCell(state_size )
    #cell = tf.contrib.rnn.DropoutWrapper(cell, output_keep_prob=keep_prob)
    cells.append(cell)
    
mul_cell = tf.nn.rnn_cell.MultiRNNCell(cells)

rnn_outputs, final_state = tf.nn.dynamic_rnn(mul_cell, rnn_inputs, dtype=tf.float32)



In [6]:
rnn_outputs_reshaped = tf.reshape(rnn_outputs,[tf.shape(rnn_outputs)[0],
                                               rnn_outputs.shape[1],1,rnn_outputs.shape[2]])
conv5 = lrelu(tf.layers.conv2d(rnn_outputs_reshaped, name="conv5", filters = 128, 
                         kernel_size = [4,1],strides = [1,1],padding = "valid", activation = None),"conv5_lrelu")

conv6 = lrelu(tf.layers.conv2d(conv5, name="conv6", filters = 256, 
                         kernel_size = [4,1],strides = [1,1],padding = "valid", activation = None),"conv6_lrelu")

pool3 = tf.layers.max_pooling2d(inputs=conv6, name = "pool3", pool_size=[2, 1], strides=[1,1])

conv7 = lrelu(tf.layers.conv2d(pool3, name="conv7", filters = 64, 
                         kernel_size = [4,1],strides = [1,1],padding = "valid", activation = None),"conv7_lrelu")

conv8 = lrelu(tf.layers.conv2d(conv7, name="conv8", filters = 128, 
                         kernel_size = [4,1],strides = [1,1],padding = "valid", activation = None),"conv8_lrelu")

pool4 = tf.layers.max_pooling2d(inputs=conv8, name = "pool4", pool_size=[2, 1], strides=[1,1])

conv9 = lrelu(tf.layers.conv2d(pool4, name="conv9", filters = 32, 
                         kernel_size = [4,1],strides = [1,1],padding = "valid", activation = None),"conv9_lrelu")

conv10 = lrelu(tf.layers.conv2d(conv9, name="conv10", filters = 64, 
                         kernel_size = [4,1],strides = [1,1],padding = "valid", activation = None),"conv10_lrelu")


pool5 = tf.layers.max_pooling2d(inputs=conv10, name = "pool5", pool_size=[4, 1], strides=[2,1])


pool5_flat = tf.reshape(pool5,[tf.shape(pool5)[0],pool5.shape[1]*pool5.shape[2]*pool5.shape[3]])


#rnn_outputs_flat = tf.reshape(rnn_outputs,[tf.shape(rnn_outputs)[0],rnn_outputs.shape[1]*rnn_outputs.shape[2]])
dense1 = tf.layers.dense(pool5_flat,512,name="dense1")
dense2 = tf.layers.dense(dense1,output_num,name="dense2")

logits = tf.squeeze(dense2,axis=1)

In [7]:
print(rnn_outputs)


Tensor("rnn/transpose:0", shape=(?, 70, 100), dtype=float32)


In [8]:
print(rnn_outputs_reshaped)
print(" ")
print(conv5)
print(conv6)
print(pool3)
print(" ")
print(conv7)
print(conv8)
print(pool4)
print(" ")
print(conv9)
print(conv10)
print(pool5)
print(" ")
print(dense1)
print(dense2)

Tensor("Reshape_1:0", shape=(?, 70, 1, 100), dtype=float32)
 
Tensor("conv5_lrelu:0", shape=(?, 67, 1, 128), dtype=float32)
Tensor("conv6_lrelu:0", shape=(?, 64, 1, 256), dtype=float32)
Tensor("pool3/MaxPool:0", shape=(?, 63, 1, 256), dtype=float32)
 
Tensor("conv7_lrelu:0", shape=(?, 60, 1, 64), dtype=float32)
Tensor("conv8_lrelu:0", shape=(?, 57, 1, 128), dtype=float32)
Tensor("pool4/MaxPool:0", shape=(?, 56, 1, 128), dtype=float32)
 
Tensor("conv9_lrelu:0", shape=(?, 53, 1, 32), dtype=float32)
Tensor("conv10_lrelu:0", shape=(?, 50, 1, 64), dtype=float32)
Tensor("pool5/MaxPool:0", shape=(?, 24, 1, 64), dtype=float32)
 
Tensor("dense1/BiasAdd:0", shape=(?, 512), dtype=float32)
Tensor("dense2/BiasAdd:0", shape=(?, 1), dtype=float32)


In [9]:


loss = tf.reduce_mean(tf.square(y - logits))
loss_sum = tf.summary.scalar('Loss', loss)

train_step = tf.train.AdamOptimizer(learning_rate = 0.0002).minimize(loss)

In [10]:

batch_size = 32

merged = tf.summary.merge_all()
saver = tf.train.Saver()
init = tf.global_variables_initializer()
sess = tf.Session()


resample="2H"

restore_checkpoint = False
save = False
tensorboard = True


if restore_checkpoint and tf.train.checkpoint_exists(checkpoint_path):
    saver.restore(sess, checkpoint_path)
else:
    sess.run(init)

In [11]:
!rm -r logs/train

rm: cannot remove 'logs/train': No such file or directory


In [12]:
data = load_data(["Calgary","London"],['2013','2014'])


Calgary Energy Balance 5min 2013
Calgary Water Balance 5min 2013
Calgary Lysimeters kg 5min2013


/notebooks/Recurrent/load_data.py:77: FutureWarning: how in .resample() is deprecated
the new syntax is .resample(...)..apply(<func>)
  'Wind': "mean",'Rain': "sum",'Lysimeter': "mean"})


London Energy Balance 5min 2013
London Water Balance 5min 2013
London Lysimeters kg 5min2013
Calgary Energy Balance 5min 2014
Calgary Water Balance 5min 2014
Calgary Lysimeters kg 5min2014
London Energy Balance 5min 2014
London Water Balance 5min 2014
London Lysimeters kg 5min2014


/usr/local/lib/python3.5/dist-packages/pandas/core/indexes/base.py:2245: FutureWarning: In the future, 'NAT < x' and 'x < NAT' will always be False.
  self._values[0] < other_diff[0]


In [13]:
n_iterations = 150000

checkpoint_path = "./logs/ET"
    
train_writer = tf.summary.FileWriter('logs/train',
                                    sess.graph)
step = 0

for i,(x_,y_) in enumerate(gen_data(batch_size,num_steps,n_iterations,
                                    data,resample=resample)):
    if(y_.shape[0] == 0):
        continue

    _,loss_,pred,summary = sess.run([train_step,loss,logits,merged],
                                    feed_dict = {x : x_, y: y_})

    if tensorboard:
        train_writer.add_summary(summary, step)

    if(i % 25 == 0):
        print("Iteration : {}  loss : {}  \nreal : {}   \npred : {}\n".format(i, loss_,
                                                                                       y_[0:3],pred[0:3]))
    step+=1
if save:
    save_path = saver.save(sess, checkpoint_path)

Iteration : 0  loss : 283.38092041015625  
real : [ 14.26952149  16.79191192  18.10677497]   
pred : [  5.27612904e-08   6.60950477e-08   4.61803893e-08]

Iteration : 25  loss : 10.448774337768555  
real : [ 13.95049164  14.44305052  15.06310846]   
pred : [ 11.09905624  11.08977413  11.09439182]

Iteration : 50  loss : 0.9785699844360352  
real : [ 16.21181     14.87755593  15.64407931]   
pred : [ 14.37176514  14.38300133  14.37356377]

Iteration : 75  loss : 25.188926696777344  
real : [ 14.08808752  16.77470599  13.83602841]   
pred : [ 12.65581608  12.64486313  12.64778519]

Iteration : 100  loss : 2.023662805557251  
real : [ 15.71140811  11.00624889  12.32250856]   
pred : [ 14.39054775  14.39042091  14.39038754]

Iteration : 125  loss : 4.1287126541137695  
real : [ 13.75033846  15.9957432   15.74559304]   
pred : [ 16.80883789  16.8179512   16.81812668]

Iteration : 150  loss : 16.108997344970703  
real : [ 17.62688407  16.10280974  13.74236414]   
pred : [ 13.80116653  13.803

Iteration : 1425  loss : 2.983933210372925  
real : [ 11.00159483  15.3110371   13.20292248]   
pred : [ 14.62243462  14.77546978  14.6341238 ]



KeyboardInterrupt: 

In [ ]:
for i,(x_,y_) in enumerate(gen_data(batch_size,num_steps,n_iterations,
                                    load_data(["Halifax"],['2013']),resample=resample)):
    if(x_.shape[0] == 0):
        continue
        
    pred = sess.run([logits], feed_dict = {x : x_ })
    
    print("Iteration : {}  \nreal : {}   \npred : {}\n".format(i, y_[0:3],pred[0][0:3]))
    if(i > 200):
        break

Halifax Energy Balance 5min 2013


In [15]:
data = next(load_data(["Calgary"],['2013']))

In [17]:
print(data)

                      Radiation      Wind   Humidity  Lysimeter       Temp  \
2013-05-16 12:00:00   16.256182  1.259000  63.416364  12.874815  10.451818   
2013-05-16 18:00:00  -49.123806  0.820819  80.275000  12.998047   7.874569   
2013-05-17 00:00:00  -62.440694  1.786375  90.811111  13.022708   3.558889   
2013-05-17 06:00:00  176.045694  1.313750  72.180972  13.000165   7.504472   
2013-05-17 12:00:00  324.389722  0.763486  36.864861  15.199888  15.742361   
2013-05-17 18:00:00  -11.024542  0.487528  37.742222  15.209717  14.858194   
2013-05-18 00:00:00  -52.335139  1.245139  69.095417  15.183530   6.657917   
2013-05-18 06:00:00  243.985583  1.778542  57.481944  15.141670  10.006958   
2013-05-18 12:00:00  397.068056  2.130014  33.206667  14.951719  17.845833   
2013-05-18 18:00:00  -20.653514  1.849778  44.300556  14.832475  14.691250   
2013-05-19 00:00:00  -25.777000  1.316125  67.887222  14.810025   9.433556   
2013-05-19 06:00:00   30.498292  1.157722  66.066250  14.860086 